# Chapter 14 RNN

In [1]:

# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "rnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf

# Basic RNN 
## Manual RNN

In [3]:
reset_graph() 

n_inputs = 3 
n_neurons = 5

X0 = tf.placeholder(tf.float32,[None,n_inputs]) 
X1 = tf.placeholder(tf.float32,[None,n_inputs]) 

Wx = tf.Variable(tf.random_normal(shape=[n_inputs,n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1,n_neurons],dtype=tf.float32)) 

Y0 = tf.tanh(tf.matmul(X0,Wx)+b) 
Y1 = tf.tanh(tf.matmul(Y0,Wy) + tf.matmul(X1,Wx) + b) 

init = tf.global_variables_initializer() 

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
import numpy as np 
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]]) 
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]]) 

with tf.Session() as sess: 
    init.run() 
    Y0_val,Y1_val = sess.run([Y0,Y1],feed_dict={X0:X0_batch,X1:X1_batch})

In [5]:
print(Y0_val)

[[-0.0664006   0.9625767   0.68105793  0.7091854  -0.898216  ]
 [ 0.9977755  -0.71978897 -0.9965761   0.9673924  -0.9998972 ]
 [ 0.99999774 -0.99898803 -0.9999989   0.9967762  -0.9999999 ]
 [ 1.         -1.         -1.         -0.99818915  0.9995087 ]]


# Using `static_rnn()`

The following code produce the same result as the previous one. 

In [6]:
n_inputs = 3
n_neurons = 5 

In [7]:
reset_graph() 
# Create input placeholders. 
X0 = tf.placeholder(tf.float32,[None,n_inputs])
X1 = tf.placeholder(tf.float32,[None,n_inputs]) 

# Creat BasicRNNCell which is like a factory that creates copies of the cell to build the unrolled RNN. 
# One for each time step. 
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons) 
# Call static_rnn() 
# givee it the cell factory, input tensors, tell it the data type of the inputs. 
output_seqs,states = tf.nn.static_rnn(basic_cell,[X0,X1],dtype=tf.float32) 
# static_rnn() returns two objects. First is a Python list containing the output tensors for each time step. 
# Second is a tensor containing final states of 
Y0,Y1 = output_seqs 

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [8]:
init = tf.global_variables_initializer() 

In [9]:
X0_batch = np.array([[0,1,2],[3,4,5],[6,7,8],[9,0,1]])
X1_batch = np.array([[9,8,7],[0,0,0],[6,5,4],[3,2,1]]) 

with tf.Session() as sess: 
    init.run() 
    Y0_val,Y1_val = sess.run([Y0,Y1],feed_dict={X0:X0_batch,X1:X1_batch})

In [10]:
Y0_val

array([[ 0.30741334, -0.32884315, -0.6542847 , -0.9385059 ,  0.52089024],
       [ 0.99122757, -0.9542541 , -0.7518079 , -0.9995208 ,  0.9820235 ],
       [ 0.9999268 , -0.99783254, -0.8247353 , -0.9999963 ,  0.99947774],
       [ 0.996771  , -0.68750614,  0.8419969 ,  0.9303911 ,  0.8120684 ]],
      dtype=float32)

## Packing Sequences 

In [11]:
n_steps = 2 
n_inputs = 3 
n_neurons = 5 

In [15]:

reset_graph() 

X = tf.placeholder(tf.float32,[None,n_steps,n_inputs]) 
# X_seqs is a Python list of n_steps tensors of shap [None,n_inputs]. 
X_seqs = tf.unstack(tf.transpose(X,perm=[1,0,2]))

basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons) 
output_seqs,states = tf.nn.static_rnn(basic_cell,X_seqs,dtype=tf.float32) 
outputs = tf.transpose(tf.stack(output_seqs),perm=[1,0,2]) 


In [16]:
init = tf.global_variables_initializer() 

In [18]:
X_batch = np.array([
    [[0,1,2],[9,8,7]],
    [[3,4,5],[0,0,0]],
    [[6,7,8],[6,5,4]],
    [[9,0,1],[3,2,1]]
])

with tf.Session() as sess: 
    init.run() 
    outputs_val = outputs.eval(feed_dict={X:X_batch})

In [19]:
print(outputs_val)

[[[-0.45652324 -0.68064123  0.40938237  0.63104504 -0.45732826]
  [-0.9428799  -0.9998869   0.94055814  0.9999985  -0.9999999 ]]

 [[-0.8001535  -0.9921827   0.7817797   0.9971031  -0.9964609 ]
  [-0.637116    0.11300932  0.5798437   0.43105596 -0.63716984]]

 [[-0.93605185 -0.9998379   0.9308867   0.9999815  -0.99998295]
  [-0.9165386  -0.9945604   0.896054    0.99987185 -0.9999751 ]]

 [[ 0.9927369  -0.9981933  -0.55543643  0.9989031  -0.9953323 ]
  [-0.02746338 -0.73191977  0.7827872   0.9525682  -0.9781773 ]]]


## Setting the sequence lengths

In [20]:
n_steps = 2
n_inputs = 3 
n_neurons = 5 

reset_graph() 

X = tf.placeholder(tf.float32,[None,n_steps,n_inputs]) 
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons) 


In [21]:
# this is new
seq_length = tf.placeholder(tf.int32,[None])
outputs,states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32,sequence_length=seq_length)

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.


In [22]:
init = tf.global_variables_initializer() 

In [23]:
X_batch = np.array([
        # step 0     step 1
        [[0, 1, 2], [9, 8, 7]], # instance 1
        [[3, 4, 5], [0, 0, 0]], # instance 2 (padded with zero vectors)
        [[6, 7, 8], [6, 5, 4]], # instance 3
        [[9, 0, 1], [3, 2, 1]], # instance 4
    ])
seq_length_batch = np.array([2, 1, 2, 2])

In [24]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run(
        [outputs, states], feed_dict={X: X_batch, seq_length: seq_length_batch})

In [25]:
print(outputs_val)

[[[-0.9123188   0.16516446  0.5548655  -0.39159346  0.20846416]
  [-1.          0.9567258   0.9983168   0.99970174  0.9651857 ]]

 [[-0.9998612   0.6702291   0.9723653   0.6631046   0.74457586]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.99999976  0.8967997   0.9986295   0.9647514   0.93662   ]
  [-0.9999526   0.9681953   0.96002865  0.98706263  0.85459226]]

 [[-0.96435434  0.99501586 -0.36150697  0.9983378   0.999497  ]
  [-0.96135855  0.9568762   0.7132288   0.97729224 -0.0958299 ]]]


## Training a sequence classifier


In [26]:
reset_graph() 

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10 

learning_rate = 0.001

X = tf.placeholder(tf.float32,[None,n_steps,n_inputs])
y = tf.placeholder(tf.int32,[None]) 

# Here we didn't define name scope. 
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units = n_neurons) 
# we didn't specifiy sequence length. 
# X: RNN inputs. Tensor of shape [Batch_size, max_time ...] 
# seq_length(optional): vector sized [batch_size]. 
outputs,states = tf.nn.dynamic_rnn(basic_cell,X,dtype=tf.float32) 

logits = tf.layers.dense(states,n_outputs) 
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y,logits=logits) 

loss = tf.reduce_mean(xentropy) 
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(loss) 
correct = tf.nn.in_top_k(logits,y,1) 
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32)) 

init = tf.global_variables_initializer() 



Instructions for updating:
Use keras.layers.dense instead.


In [27]:

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [28]:

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [30]:
X_test = X_test.reshape((-1, n_steps, n_inputs))

In [ ]:
n_epochs = 100 
batch_size = 150 

with tf.Session() as sess: 
    init.run() 
    for epoch in range(n_epochs): 
        for X_batch,y_batch in shuffle_batch(X_train,y_train,batch_size): 
            X_batch = X_batch.reshape((-1,n_steps,n_inputs))
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})
        acc_batch = accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test  = accuracy.eval(feed_dict={X:X_test,y:y_test})
        print(epoch,"Last batch accuracy:",acc_batch,"Test accuracy:",acc_test)

0 Last batch accuracy: 0.9533333 Test accuracy: 0.9288
1 Last batch accuracy: 0.96 Test accuracy: 0.9471
2 Last batch accuracy: 0.96 Test accuracy: 0.9499
3 Last batch accuracy: 0.96 Test accuracy: 0.9566
4 Last batch accuracy: 0.96666664 Test accuracy: 0.9644
5 Last batch accuracy: 0.94 Test accuracy: 0.965
6 Last batch accuracy: 0.98 Test accuracy: 0.9674
7 Last batch accuracy: 0.96 Test accuracy: 0.9691
8 Last batch accuracy: 0.9866667 Test accuracy: 0.974
9 Last batch accuracy: 0.99333334 Test accuracy: 0.9746
10 Last batch accuracy: 0.97333336 Test accuracy: 0.9752
11 Last batch accuracy: 0.9866667 Test accuracy: 0.9766
12 Last batch accuracy: 0.9533333 Test accuracy: 0.965
13 Last batch accuracy: 0.9866667 Test accuracy: 0.9767
14 Last batch accuracy: 0.99333334 Test accuracy: 0.9726
15 Last batch accuracy: 0.9866667 Test accuracy: 0.975
16 Last batch accuracy: 0.9866667 Test accuracy: 0.9657
17 Last batch accuracy: 0.9866667 Test accuracy: 0.9766
18 Last batch accuracy: 0.986666

## Time series


In [ ]:
t_min,t_max = 0,30
resolution = 0.1

def time_series(t): 
    return t*np.sin(t)/3+2*np.sin(t*5)

def next_batch(batch_size,n_steps): 
    t0 = np.random.rand(batch_size,1)*(t_max-t_min-n_steps*resolution) 
    Ts = t0 + np.arange(0.,n_steps+1) * resolution
    ys = time_series(Ts)
    return ys[:,:-1].reshape(-1,n_steps,1),ys[:,1:].reshape(-1,n_steps,1) 

In [ ]:
t = np.linspace(t_min,t_max,int((t_max-t_min)/resolution)) 

n_steps = 20
t_instance = np.linspace(12.2,12.2+resolution * (n_steps + 1),n_steps+1) 

plt.figure(figsize=(11,4))
plt.subplot(121) 
plt.title("A time series(generated)",fontsize=14) 
plt.plot(t,time_series(t),label = r"$t.\sin(t)/3+2.\sin(5t)$")
plt.plot(t_instance[:-1],time_series(t_instance[:-1]),"b-",linewidth=3,label="A training instance") 
plt.legend(loc="lower left", fontsize=14)
plt.axis([0,30,-17,13]) 
plt.xlabel("Time") 
plt.ylabel("Value") 

plt.subplot(122)
plt.title("A training instance", fontsize=14)
plt.plot(t_instance[:-1], time_series(t_instance[:-1]), "bo", markersize=10, label="instance")
plt.plot(t_instance[1:], time_series(t_instance[1:]), "w*", markersize=10, label="target")
plt.legend(loc="upper left")
plt.xlabel("Time")


save_fig("time_series_plot")
plt.show()

In [ ]:
reset_graph()

n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])